Make NumPy available:

In [6]:
import numpy as np

## Exercise 07.1 (indexing and timing)

Create two very long NumPy arrays `x` and `y` and sum the arrays using:

1. The NumPy addition syntax, `z = x + y`; and
2. A `for` loop that computes the sum entry-by-entry

Compare the time required for the two approaches for vectors of different lengths (use a very long vector for 
the timing). The values of the array entries are not important for this test. Use `%time` to report the time.

*Hint:* To loop over an array using indices, try a construction like:

In [7]:
x = np.ones(1000000)
y = np.ones(x.size)
for i in range(len(x)):
    x[i]*y[i]

#### (1) Add two vectors using built-in addition operator:

In [8]:
# YOUR CODE HERE
def addvectors(x,y):
  return x + y

#### (2) Add two vectors using own implementation:

In [9]:
# YOUR CODE HERE
def addvectors_slow(x,y):
  z = []
  for ix,iy in zip(x,y):
    z.append( ix + iy ) 
  return z

print(" built in method:")
%time z = addvectors(x,y)
print("\n my slow method:")
%time z = addvectors_slow(x,y)


 built in method:
CPU times: user 965 µs, sys: 3.95 ms, total: 4.92 ms
Wall time: 5.67 ms

 my slow method:
CPU times: user 373 ms, sys: 48.5 ms, total: 422 ms
Wall time: 426 ms


### Optional extension: just-in-time (JIT) compilation

You will see a large difference in the time required between your NumPy and 'plain' Python implementations. This is due to Python being an *interpreted* language as opposed to a *compiled* language. A way to speed up plain Python implementions is to convert the interpreted Python code into compiled code. A tool for doing this is [Numba](https://numba.pydata.org/).

Below is an example using Numba and JIT to accelerate a computation:

In [10]:
!pip -q install numba 
import numba
import math

def compute_sine_native(x):
    z = np.zeros(len(x))
    for i in range(len(z)):
        z[i] = math.sin(x[i])
    return z

@numba.jit
def compute_sine_jit(x):
    z = np.zeros(len(x))
    for i in range(len(z)):
        z[i] = math.sin(x[i])
    return z
    
x = np.ones(10000000)
%time z = compute_sine_native(x)
compute_sine_jit(x)
%time z = compute_sine_jit(x)

CPU times: user 3.64 s, sys: 39.8 ms, total: 3.68 s
Wall time: 3.68 s
CPU times: user 166 ms, sys: 0 ns, total: 166 ms
Wall time: 166 ms


**Task:** Test if Numba can be used to accelerate your implementation that uses indexing to sum two arrays, and by how much.

In [11]:
## YOUR CODE HERE

@numba.jit
def addvectors_JIT(x,y):
  return x + y

@numba.jit
def addvectors_JIT(x,y):
  z = []
  for ix,iy in zip(x,y):
    z.append( ix + iy ) 
  return z


x = np.ones(1000)
y = np.ones(x.size)

print(" JIT compiled built in method:")
%time z = addvectors_JIT(x,y)
print("\n JIT compiled original slow method:")
%time z = addvectors_JIT(x,y)

# JIT compiled built in method is a lot slower

# JIT compiled iterative method is about 9x faster

 JIT compiled built in method:
CPU times: user 154 ms, sys: 3.48 ms, total: 157 ms
Wall time: 158 ms

 JIT compiled original slow method:
CPU times: user 98 µs, sys: 0 ns, total: 98 µs
Wall time: 101 µs


## Exercise 07.2 (member functions and slicing)

Anonymised scores (out of 60) for an examination are stored in a NumPy array. Write:

1. A function that takes a NumPy array of the raw scores and returns the scores as percentages, sorted from 
   lowest to highest (try using `scores.sort()`, where `scores` is a NumPy array holding the scores).
1. A function that returns the maximum, minimum and mean of the raw scores as a dictionary with the 
   keys '`min`', '`max`' and '`mean`'. Use the NumPy array functions `min()`, `max()` and `mean()` to do the 
   computation, e.g. `max = scores.max()`.  
   
   Design your function for the min, max and mean to optionally exclude the highest and lowest scores from the 
   computation of the min, max and mean. 
   
   *Hint:* sort the array of scores and use array slicing to exclude
   the first and the last entries.

Use the scores 
```python
scores = np.array([58.0, 35.0, 24.0, 42, 7.8])
```
to test your functions.

In [12]:
def to_percentage_and_sort(scores):
    # YOUR CODE HERE
    return np.sort(scores*100/60)

def statistics(scores, exclude=False):
    # YOUR CODE HERE

    sorted_scores = np.sort(scores)

    if exclude:
      sorted_scores = sorted_scores[1:-1]

    min = sorted_scores[0]
    max = sorted_scores[-1]
    mean = sorted_scores.mean()

    return {"min":min, "max":max, "mean":mean}
  
scores = np.array([58.0, 35.0, 24.0, 42, 7.8])
to_percentage_and_sort(scores)

array([13.        , 40.        , 58.33333333, 70.        , 96.66666667])

In [13]:
scores = np.array([58.0, 35.0, 24.0, 42, 7.8])
assert np.isclose(to_percentage_and_sort(scores), [ 13.0, 40.0, 58.33333333,  70.0, 96.66666667]).all()

s0 = statistics(scores)

assert round(s0["min"] - 7.8, 10) == 0.0
assert round(s0["mean"] - 33.36, 10) == 0.0
assert round(s0["max"] - 58.0, 10) == 0.0

s1 = statistics(scores, True)
assert round(s1["min"] - 24.0, 10) == 0.0
assert round(s1["mean"] - 33.666666666666666667, 10) == 0.0
assert round(s1["max"] - 42.0, 10) == 0.0

## Exercise 07.3 (slicing)

For the two-dimensional array

In [14]:
A = np.array([[4.0, 7.0, -2.43, 67.1],
             [-4.0, 64.0, 54.7, -3.33],
             [2.43, 23.2, 3.64, 4.11],
             [1.2, 2.5, -113.2, 323.22]])
print(A)

[[   4.      7.     -2.43   67.1 ]
 [  -4.     64.     54.7    -3.33]
 [   2.43   23.2     3.64    4.11]
 [   1.2     2.5  -113.2   323.22]]


use array slicing for the below operations, printing the results to the screen to check. Try to use array slicing such that your code would still work if the dimensions of `A` were enlarged.



#### 1. Extract the third column as a 1D array

In [15]:
# YOUR CODE HERE
A[:,2]

array([  -2.43,   54.7 ,    3.64, -113.2 ])

#### 2. Extract the first two rows as a 2D sub-array

In [22]:
# YOUR CODE HERE
A[:2,:]

array([[ 4.  ,  7.  , -2.43, 67.1 ],
       [-4.  , 64.  , 54.7 , -3.33]])

#### 3.  Extract the bottom-right $2 \times 2$ block as a 2D sub-array

In [17]:
# YOUR CODE HERE
A[-2:,-2:]

array([[   3.64,    4.11],
       [-113.2 ,  323.22]])

#### 4. Sum the last column

In [26]:
# YOUR CODE HERE

np.sum(A[:,-1])

391.1

#### Compute transpose

Compute the transpose of `A` (search online to find the function/syntax to do this).

In [19]:
# YOUR CODE HERE
A.T

array([[   4.  ,   -4.  ,    2.43,    1.2 ],
       [   7.  ,   64.  ,   23.2 ,    2.5 ],
       [  -2.43,   54.7 ,    3.64, -113.2 ],
       [  67.1 ,   -3.33,    4.11,  323.22]])

## Exercise 07.4 (optional extension)

In a previous exercise you implemented the bisection algorithm to find approximate roots of a mathematical function. Use the SciPy bisection function `optimize.bisect` (https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html) to find roots of the mathematical function that was used in the previous exercise. Compare the results computed by SciPy and your program from the earlier exercise, and compare the computational time (using `%time`).

In [20]:
from scipy import optimize

# YOUR CODE HERE
def my_f(x):
    "Evaluate polynomial function"
    return x**3 - 6*x**2 + 4*x + 12




def compute_root(f, x0, x1, tol, max_it, it_count=0):

    x_mid = (x0 + x1)/2
    f0 = f(x0)
    f_mid = f(x_mid)
    it_count +=1
    error = abs(f_mid)

    if  it_count > max_it:
      print("max iterations reached")
      raise Exception

    if error < tol:
      return x_mid, f_mid, it_count

    if f0 * f_mid < 0:
      return compute_root(f, x0, x_mid, tol, max_it, it_count) # sign change beween x0 and x_mid : x1 becomes x_mid
    else:
      return compute_root(f, x_mid, x1, tol, max_it, it_count) # sign change between x_mid adn x1 : x0 becomes x_mid

%time compute_root(my_f, x0=3, x1=6, tol=1e-12, max_it=1000)     
%time optimize.bisect(my_f, 3, 6, xtol=1e-12)  # over 2x faster


CPU times: user 98 µs, sys: 10 µs, total: 108 µs
Wall time: 110 µs
CPU times: user 55 µs, sys: 6 µs, total: 61 µs
Wall time: 63.4 µs


4.5340701967222685